<a href="https://colab.research.google.com/github/Srinivas2109/testbot/blob/main/gemma_connection_ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install git+https://github.com/huggingface/transformers -U
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes
!pip install trl
!pip install git+https://github.com/huggingface/peft.git

In [ ]:
!pip install -i https://pypi.org/simple/ bitsandbytes

In [ ]:
from huggingface_hub import notebook_login
notebook_login(write_permission=True)

In [ ]:
import torch
import os
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Define conversation template
chat_template = """
**You:** {user_input}
**Gemma-7b:** {model_response}
"""

# Load pre-trained model and tokenizer
#model_id = "google/gemma-2b-it"
#lora_config = BitsAndBytesConfig(
#    load_in_4bit=True,
 #   bnb_4bit_quant_type="nf4",
  #  bnb_4bit_compute_dtype=torch.bfloat16
#)
quantization_config = BitsAndBytesConfig(load_in_8bit=True)

tokenizer = AutoTokenizer.from_pretrained("google/gemma-7b-it")
model = AutoModelForCausalLM.from_pretrained("google/gemma-7b-it", quantization_config=quantization_config)



# tokenizer = AutoTokenizer.from_pretrained(model_id)
# quantized_model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=lora_config, device_map={"":0})

In [ ]:
import warnings
warnings.filterwarnings('ignore')

def chatbot():
  """
  Interactive chat function with Gemma-2b chatbot.
  """
  while True:
    user_input = input("You: ")
    if user_input.lower() == "quit":
        break

    # Encode user input with tokenizer
   # input_ids = tokenizer.encode(user_input, return_tensors="pt")

    # Generate model response

    input_ids = tokenizer(user_input, return_tensors="pt").to("cuda")
    outputs = model.generate(**input_ids)
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))
    # output = quantized_model.generate(input_ids, max_length=1024, do_sample=True, top_k=10, top_p=0.9)
    # model_response = tokenizer.decode(output[0], skip_special_tokens=True)

    # Print conversation

if __name__ == "__main__":
    print("Welcome to the Gemma-7b Chatbot!")
    chatbot()
    print("Goodbye!")